# <center>**TOÁN ỨNG DỤNG VÀ THỐNG KÊ**</center>
## <center>**Project 3 - Đồ án Diagonalizable Matrix**</center>

***
## **1. Thông tin sinh viên**
- Họ và tên: Nguyễn Thái Bảo
- Mã số sinh viên: 23120023
- Lớp: 23CTT1
- Email: 23120023@student.hcmus.edu.vn

***
## **2. Giải thuật chéo hóa ma trận vuông A**


- Bước 1. Tìm đa thức đặc trưng det(A − λI). Nếu PA(λ) có tổng các lũy thừa khác n thì A không chéo hóa được và thuật toán kết thúc, ngược lại chuyển sang bước 2.

- Bước 2. Tìm tất cả các nghiệm λi của phương trình đa thức đặc trưng. Với mỗi trị riêng λi tìm cơ sở và số chiều cho không gian nghiệm phương trình (A − λIn)X = 0.
 Nếu mỗi λi số chiều không gian nghiệm nhỏ hơn lũy thừa của λi trong đa thức đặc trưng thì A không chéo hóa được và thuật toán kết thúc, ngược lại chuyển sang bước 3.

- Bước 3. Với các vector trong cơ sở không gian nghiệm tìm được ở bước 2, ta đặt ma trận P là ma trận có được bằng cách dựng các vector thành các cột. Khi đó ma trận P làm chéo A và P^(-1)AP là ma trận đường chéo. diag(λ1, ..., λr).



***
## **3. Các hàm tiện ích (Utils)**

In [67]:
def create_matrix(rows, cols, default_value = 0):
    """Tạo ma trận với rows hàng và cols cột"""
    return [[default_value for _ in range(cols)] for _ in range(rows)]

def create_identity_matrix(n):
    """Tạo ma trận đơn vị"""
    identity = create_matrix(n, n)
    for i in range(n):
        identity[i][i] = 1
    return identity

def matrix_copy(A):
    """Tạo bản sao của ma trận A"""
    return [row[:] for row in A]

def matrix_multiply(A, B):
    """Nhân hai ma trận A và B"""
    if len(A[0]) != len(B):
        raise ValueError("Số cột của A phải bằng số hàng của B")

    m = len(A)
    n = len(A[0])
    p = len(B[0])
    C = create_matrix(m, p)
    
    for i in range(m):
        for j in range(p):
            for k in range(n):
                C[i][j] += A[i][k] * B[k][j]
    
    return C

def matrix_subtract(A, B):
    """Trừ hai ma trận A và B"""
    m = len(A)
    n = len(A[0])
    C = create_matrix(m, n)
    
    for i in range(m):
        for j in range(n):
            C[i][j] = A[i][j] - B[i][j]
    
    return C

def num_multiply(num, A):
    """Nhân ma trận A với một số"""
    m = len(A)
    n = len(A[0])
    C = create_matrix(m, n)
    
    for i in range(m):
        for j in range(n):
            C[i][j] = num * A[i][j]
    
    return C

def matrix_subtract_lambda_I(A, lambda_val):
    """Tạo ma trận A - lambda*I"""
    n = len(A)
    result = matrix_copy(A)
    
    for i in range(n):
        result[i][i] -= lambda_val
    
    return result

def polynomial_multiply(p1, p2):
    """Nhân hai đa thức"""
    deg1 = len(p1) - 1
    deg2 = len(p2) - 1
    result = [0] * (deg1 + deg2 + 1)
    
    for i in range(len(p1)):
        for j in range(len(p2)):
            result[i + j] += p1[i] * p2[j]
    
    return result

def determinant(A):
    """Tính định thức của ma trận A bằng phương pháp khai triển theo cột"""
    n = len(A)
    
    # Trường hợp cơ bản
    if n == 1:
        return A[0][0]
    if n == 2:
        return A[0][0] * A[1][1] - A[0][1] * A[1][0]
    
    det = 0
    for j in range(n):
        # Tạo ma trận con bằng cách loại bỏ hàng 0 và cột j
        minor = create_matrix(n-1, n-1)
        for i in range(1, n):
            minor_col = 0
            for k in range(n):
                if k != j:
                    minor[i-1][minor_col] = A[i][k]
                    minor_col += 1
        
        cofactor = A[0][j] * ((-1) ** j) * determinant(minor) 
        det += cofactor
    
    return det

def round_matrix(A, decimals = 10):
    """Làm tròn các phần tử của ma trận"""
    result = []
    for row in A:
        result.append([round(x, decimals) for x in row])
    return result

def print_matrix(A):
    """In ma trận A"""
    for row in A:
        print([round(x, 10) for x in row])

---
## **4. Hàm chéo hóa ma trận (Diagonalize matrix)**

Cho A là ma trận có thể chéo hóa được. Viết chương trình tìm ma trận chéo P, P^-1 và ma trận đường chéo D, biết rằng A = PDP^(-1). 
Lưu ý:  
- Phải sử dụng thuật toán chéo hóa đã được hướng dẫn trong phần lý thuyết và bài tập. Không được dùng các hàm có sẵn của các thư viện để chéo hóa. 
- Để tìm ma trận nghịch đảo P^(-1), sử dụng thuật toán Gauss - Jordan đã được hướng dẫn trong phần lý thuyết và bài tập, không được dùng các hàm có sẵn của các thư viện để tìm ma trận nghịch đảo. 

### Tính đa thức đặc trưng P(λ) = det(A - λI) theo thứ tự lũy thừa giảm dần

In [68]:
def get_minor_matrix(matrix, rows_to_remove, cols_to_remove):
    """Trả về ma trận con sau khi loại bỏ các hàng và cột nhất định"""
    n = len(matrix)
    minor = []
    
    for i in range(n):
        if i not in rows_to_remove:
            row = []
            for j in range(n):
                if j not in cols_to_remove:
                    row.append(matrix[i][j])
            minor.append(row)
    
    return minor

In [69]:
def characteristic_polynomial(A):
    n = len(A)
    
    # Với n = 1, P(λ) = a - λ
    if n == 1:
        return [-1, A[0][0]]
    
    # Với n = 2, P(λ) = λ^2 - trace(A)λ + det(A), dùng vết của ma trận
    if n == 2:
        trace = A[0][0] + A[1][1]
        det_A = A[0][0] * A[1][1] - A[0][1] * A[1][0]
        return [1, -trace, det_A]
    
    # Với n = 3, tính trực tiếp
    if n == 3:
        a, b, c = A[0][0], A[0][1], A[0][2]
        d, e, f = A[1][0], A[1][1], A[1][2]
        g, h, i = A[2][0], A[2][1], A[2][2]
        
        # P(λ) = -λ^3 + trace(A)λ^2 - minor_sum(λ) + det(A)
        trace = a + e + i     # Vết của ma trận
        
        # Tổng các định thức minor 2x2 chính
        minor_sum = (a*e + a*i + e*i) - (b*d + c*g + f*h)
        
        # Định thức của A
        det_A = a*e*i + b*f*g + c*d*h - (c*e*g + b*d*i + a*f*h)     # quy tắc Sarrus
        
        return [-1, trace, -minor_sum, det_A]
    
    # Với n > 3, dùng phương pháp khai triển
    coeffs = [0] * (n + 1)
    coeffs[0] = (-1)**n  # Hệ số của λ^n
    
    for k in range(1, n+1):
        sign = (-1)**(n-k)
        sum_det = 0

        # Trường hợp đơn giản: k = n -> toàn bộ ma trận
        if k == n:
            sum_det = determinant(A)

        # Trường hợp k = 1: tính tổng các phần tử đường chéo (trace)
        elif k == 1:
            sum_det = sum(A[i][i] for i in range(n))

        # Trường hợp k = 0: hệ số của λ^n là (-1)^n
        elif k == 0:
            sum_det = 1
            
        else:
            # Trường hợp phức tạp hơn: cần phải xem xét tất cả các minor
            for i in range(n):
                for j in range(n):
                    # Tạo ma trận con bằng cách loại bỏ hàng i và cột j
                    minor = get_minor_matrix(A, [i], [j])
                    # Tính định thức của ma trận con
                    sub_det = determinant(minor)
                    # Cộng vào tổng với hệ số tương ứng
                    sum_det += A[i][j] * sub_det
            
        coeffs[k] = sign * sum_det
        
    return coeffs

### Tìm trị riêng từ đa thức đặc trưng

In [70]:
def find_eigenvalues(poly):
    
    # Loại bỏ hệ số 0 ở đầu đa thức
    while len(poly) > 0 and abs(poly[0]) < 1e-10:
        poly = poly[1:]
    
    # Nếu đa thức rỗng hoặc hằng số, không có nghiệm
    if len(poly) <= 1:
        return {}
    
    # Chuẩn hóa đa thức để hệ số cao nhất là 1
    poly = [coef / poly[0] for coef in poly]
    
    # Tìm nghiệm của đa thức
    roots_with_multiplicity = find_polynomial_roots_with_multiplicity(poly)
    
    # Làm tròn nghiệm để tránh sai số số học
    eigenvalues = {}
    for root, multiplicity in roots_with_multiplicity:
        # Làm tròn số thực để tránh sai số
        if abs(root - round(root)) < 1e-10:
            root = round(root)
        eigenvalues[root] = multiplicity
    
    print("Trị riêng và bội số tương ứng:", eigenvalues)
    return eigenvalues

def find_polynomial_roots_with_multiplicity(poly, tol = 1e-10):
    """Tìm nghiệm thực của đa thức kèm theo bội số"""
    n = len(poly) - 1  # Bậc của đa thức
    
    # Với đa thức bậc 1: ax + b = 0
    if n == 1:
        root = -poly[1] / poly[0]
        return [(root, 1)]
    
    # Với đa thức bậc 2: ax^2 + bx + c = 0, dùng công thức nghiệm
    elif n == 2:
        a, b, c = poly
        delta = b**2 - 4*a*c
        
        if abs(delta) < 1e-10:  # Delta gần 0 -> nghiệm kép
            root = -b / (2*a)
            return [(root, 2)]
        elif delta > 0:
            sqrt_delta = delta**0.5
            root1 = (-b + sqrt_delta) / (2*a)
            root2 = (-b - sqrt_delta) / (2*a)
            return [(root1, 1), (root2, 1)]
        else:
            return []   # Không có nghiệm thực, ta xem như vô nghiệm
    
    # Với đa thức bậc 3 trở lên, sử dụng phương pháp tìm nghiệm và xác định bội
    else:
        roots_with_multiplicity = []
        working_poly = poly.copy()
        
        while len(working_poly) > 1:  # Đa thức có bậc ít nhất là 1
            # Tìm một nghiệm
            roots = find_real_roots_with_bisection(working_poly)
            if not roots:
                break  # Không tìm thấy nghiệm thực nào khác
            
            root = roots[0]  # Lấy một nghiệm đã tìm được
            
            # Xác định bội của nghiệm bằng cách kiểm tra đạo hàm
            multiplicity = 0
            current_poly = working_poly.copy()
            
            while len(current_poly) > 1:
                # Kiểm tra nếu root là nghiệm của đa thức hiện tại
                value = evaluate_polynomial(current_poly, root)
                if abs(value) > tol:
                    break
                
                multiplicity += 1
                current_poly = derivative_polynomial(current_poly)
            
            # Thêm nghiệm và bội số vào kết quả
            roots_with_multiplicity.append((root, multiplicity))
            
            # Giảm bậc đa thức bằng cách chia cho (x - root) ^ bội
            for _ in range(multiplicity):
                working_poly = polynomial_deflation(working_poly, root)
        
        return roots_with_multiplicity

def polynomial_deflation(poly, root):
    """Chia đa thức cho (x - nghiệm) bằng thuật toán Horner"""
    n = len(poly)
    if n <= 1:
        return []
    
    result = [poly[0]]
    for i in range(1, n - 1):
        result.append(poly[i] + result[i-1] * root)
    
    # Kiểm tra phần dư (remainder) gần 0
    remainder = poly[n-1] + result[n-2] * root
    if abs(remainder) > 1e-8:
        print(f"Cảnh báo: Phần dư khi chia đa thức cho (x - {root}) không gần 0: {remainder}")
    
    return result

def evaluate_polynomial(poly, x):
    """Tính giá trị của đa thức tại x"""
    result = 0
    for coef in poly:
        result = result * x + coef
    return result

def find_real_roots_with_bisection(poly, range_min = -100, range_max = 100, steps = 1000, tol = 1e-10):
    """Tìm nghiệm thực của đa thức bằng phương pháp chia đôi"""
    roots = []
    step_size = (range_max - range_min) / steps
    
    # Tìm các khoảng có thể chứa nghiệm
    x_prev = range_min
    y_prev = evaluate_polynomial(poly, x_prev)
    
    for i in range(1, steps + 1):
        x_curr = range_min + i * step_size
        y_curr = evaluate_polynomial(poly, x_curr)
        
        # Kiểm tra xem có đổi dấu hay không
        if y_prev * y_curr <= 0:
            # Tìm nghiệm chính xác hơn bằng phương pháp chia đôi
            root = bisection_method(poly, x_prev, x_curr, tol)

            # Kiểm tra xem nghiệm này đã có trong danh sách chưa
            is_duplicate = False
            for existing_root in roots:
                if abs(root - existing_root) < tol:
                    is_duplicate = True
                    break
            if not is_duplicate:
                roots.append(root)
        
        x_prev = x_curr
        y_prev = y_curr
    
    # Tìm nghiệm dùng phương pháp Newton để tăng độ chính xác
    refined_roots = []
    for root in roots:
        refined_root = newton_method(poly, root, tol)
        # Kiểm tra xem nghiệm tinh chỉnh đã có trong danh sách chưa
        is_duplicate = False
        for existing_root in refined_roots:
            if abs(refined_root - existing_root) < tol:
                is_duplicate = True
                break
        if not is_duplicate:
            refined_roots.append(refined_root)
    
    return refined_roots

def bisection_method(poly, a, b, tol = 1e-10, max_iter = 100):
    """Tìm nghiệm của đa thức trong khoảng [a, b] bằng phương pháp chia đôi"""
    fa = evaluate_polynomial(poly, a)
    fb = evaluate_polynomial(poly, b)
    
    # Kiểm tra xem khoảng có chứa nghiệm không
    if fa * fb > 0:
        # Không có nghiệm trong khoảng hoặc có số chẵn nghiệm
        return (a + b) / 2
    
    # Nếu một trong hai đầu mút là nghiệm
    if abs(fa) < tol:
        return a
    if abs(fb) < tol:
        return b
    
    # Thực hiện phương pháp chia đôi
    for _ in range(max_iter):
        c = (a + b) / 2
        fc = evaluate_polynomial(poly, c)
        
        if abs(fc) < tol:
            return c
        
        if fa * fc < 0:
            b = c
            fb = fc
        else:
            a = c
            fa = fc
        
        # Điều kiện dừng
        if (b - a) < tol:
            return (a + b) / 2
    
    return (a + b) / 2

def newton_method(poly, x0, tol=1e-10, max_iter=100):
    """Tìm nghiệm của đa thức bằng phương pháp Newton"""
    x = x0
    
    for _ in range(max_iter):
        # Tính giá trị của đa thức và đạo hàm tại x
        f_x = evaluate_polynomial(poly, x)
        
        if abs(f_x) < tol:
            return x
        
        # Tính đạo hàm
        df_x = 0
        for i in range(len(poly) - 1):
            df_x = df_x * x + (len(poly) - i - 1) * poly[i]
        
        if abs(df_x) < tol:  # Tránh chia cho 0
            break
        
        # Áp dụng công thức Newton
        x_new = x - f_x / df_x
        
        # Kiểm tra hội tụ
        if abs(x_new - x) < tol:
            return x_new
        
        x = x_new
    
    return x

def derivative_polynomial(poly):
    """Tính đạo hàm của đa thức"""
    n = len(poly)
    if n <= 1:
        return [0]
    
    derived = []
    for i in range(n - 1):
        derived.append((n - i - 1) * poly[i])
    
    return derived

### Tìm các vector riêng tương ứng với trị riêng

In [71]:
def gauss_elimination(A):
    """Đưa ma trận A về dạng bậc thang rút gọn"""
    A = [row[:] for row in A]  # Copy ma trận
    m = len(A)
    n = len(A[0])
    
    h = 0  # Chỉ số cur_row_pivot
    k = 0  # Chỉ số cur_col_pivot
    pivot_positions = [] 
    
    while h < m and k < n:
        # Tìm pivot trong cột k
        max_val = 0
        max_row = h
        for i in range(h, m):
            if abs(A[i][k]) > max_val:
                max_val = abs(A[i][k])
                max_row = i
        
        if abs(A[max_row][k]) < 1e-10:  # Gần như bằng 0
            # Không có pivot trong cột này
            k += 1
            continue
        
        # Hoán đổi hàng
        if max_row != h:
            A[h], A[max_row] = A[max_row], A[h]
        
        # Lưu vị trí pivot
        pivot_positions.append(k)
        
        # Chuẩn hóa hàng pivot -> biến phần tử pivot thành 1
        pivot = A[h][k]
        for j in range(k, n):
            A[h][j] /= pivot
        
        # Khử tất cả các hàng khác (trên và dưới) để cột pivot chỉ có 1 phần tử khác 0
        for i in range(m):
            if i != h:
                factor = A[i][k]
                for j in range(k, n):
                    A[i][j] -= factor * A[h][j]
        
        h += 1
        k += 1
    
    for i in range(m):
        for j in range(n):
            if abs(A[i][j]) < 1e-10:
                A[i][j] = 0.0
    
    return A, pivot_positions

def find_eigenvectors(A, eigenvalue, multiplicity):
    """Tìm vector riêng tương ứng với trị riêng"""
    n = len(A)
    A_lambda = matrix_subtract_lambda_I(A, eigenvalue)
    
    # Đưa A_lambda về dạng bậc thang rút gọn
    rref_matrix, pivot_positions = gauss_elimination(A_lambda)
    
    # Tìm các biến tự do
    free_vars = [j for j in range(n) if j not in pivot_positions]
    
    # Số chiều không gian nghiệm
    dim = len(free_vars)
    
    print(f"Trị riêng {eigenvalue} có bội {multiplicity} và không gian nghiệm có số chiều {dim}")
    
    if dim < multiplicity:
        return None  # Ma trận không chéo hóa được
    
    # Tìm cơ sở của không gian nghiệm
    basis = []
    
    for free_var in free_vars:
        # Tạo vector với biến tự do được gán giá trị 1
        vec = [0] * n
        vec[free_var] = 1.0
        
        # Thay thế ngược từ dưới lên để tìm các giá trị còn lại
        for i in range(len(pivot_positions) - 1, -1, -1):
            row = i
            col = pivot_positions[i]
            
            # Tính tổng các thành phần đã biết
            sum_val = 0
            for j in range(col + 1, n):
                sum_val += rref_matrix[row][j] * vec[j]
            
            # Tính giá trị của biến ở vị trí pivot
            vec[col] = -sum_val / rref_matrix[row][col]
        
        basis.append(vec)
    
    return basis

### Tính ma trận nghịch đảo P^(-1) bằng phương pháp Gauss-Jordan

In [ ]:
def compute_inverse(P):

    n = len(P)
    
    # Tạo ma trận mở rộng [P | I]
    P_augmented = []
    for i in range(n):
        row = P[i][:]
        for j in range(n):
            if i == j:
                row.append(1.0)
            else:
                row.append(0.0)
        P_augmented.append(row)
    
    # Biến đổi Gauss - Jordan
    for i in range(n):
        # Tìm pivot
        max_val = abs(P_augmented[i][i])
        max_row = i
        for k in range(i + 1, n):
            if abs(P_augmented[k][i]) > max_val:
                max_val = abs(P_augmented[k][i])
                max_row = k
        
        # Hoán đổi hàng
        if max_row != i:
            P_augmented[i], P_augmented[max_row] = P_augmented[max_row], P_augmented[i]
        
        # Chuẩn hóa hàng pivot
        pivot = P_augmented[i][i]
        for j in range(i, 2*n):
            P_augmented[i][j] /= pivot
        
        # Khử tất cả các hàng khác
        for k in range(n):
            if k != i:
                factor = P_augmented[k][i]
                for j in range(i, 2*n):
                    P_augmented[k][j] -= factor * P_augmented[i][j]
    
    # Lấy ma trận nghịch đảo
    P_inverse = []
    for i in range(n):
        P_inverse.append(P_augmented[i][n:])
    
    return P_inverse

### Kiểm tra tính chéo hóa của ma trận, trả về ma trận P, P^(-1) và D nếu chéo hóa được

In [ ]:
def is_diagonalizable(A):

    print("Bước 1: Tính đa thức đặc trưng")
    char_poly = characteristic_polynomial(A)
    print(f"Hệ số của đa thức đặc trưng (theo lũy thừa tăng dần): {char_poly}")
    
    print("\nBước 2: Tìm trị riêng và vector riêng")
    n = len(A)
    eigenvalues = find_eigenvalues(char_poly)
    if sum(eigenvalues.values()) != n:
        print("Ma trận không chéo hóa được: Tổng các lũy thừa khác n")
        return None, None, None
    
    # Tìm vector riêng cho mỗi trị riêng
    eigenvectors = {}
    for eigenval, multiplicity in eigenvalues.items():
        basis = find_eigenvectors(A, eigenval, multiplicity)
        if basis is None:
            print("Ma trận không chéo hóa được: Số chiều không gian nghiệm < bội của trị riêng")
            return None, None, None
        eigenvectors[eigenval] = basis
    
    print("\nBước 3: Xây dựng ma trận P và D")
    P = create_matrix(n, n)
    D = create_matrix(n, n)
    
    col = 0
    for eigenval, vectors in eigenvectors.items():
        for v in vectors:
            for i in range(n):
                P[i][col] = v[i]
            D[col][col] = eigenval
            col += 1
    
    # Tính P^-1
    P_inverse = compute_inverse(P)
    
    return P, P_inverse, D

***
## **5. Kiểm tra kết quả**

Ví dụ 1:

In [74]:
A = [[1, 3, 3], [-3, -5, -3], [3, 3, 1]]

print("Ma trận A:")
print_matrix(A)
print()

P, P_inverse, D = is_diagonalizable(A)

if P is not None:
    print("Ma trận P (các cột là vector riêng):")
    print_matrix(P)
    
    print("\nMa trận P^-1:")
    print_matrix(P_inverse)
    
    print("\nMa trận đường chéo D:")
    print_matrix(D)
    
    print("\nKiểm tra ma trận A = P*D*P^-1:")
    result = matrix_multiply(P, matrix_multiply(D, P_inverse))
    print_matrix(result)
else:
    print("\nMa trận không thể chéo hóa được.")

Ma trận A:
[1, 3, 3]
[-3, -5, -3]
[3, 3, 1]

Bước 1: Tính đa thức đặc trưng
Hệ số của đa thức đặc trưng (theo lũy thừa tăng dần): [-1, -3, 0, 4]

Bước 2: Tìm trị riêng và vector riêng
Trị riêng và bội số tương ứng: {-2: 2, 1: 1}
Trị riêng -2 có bội 2 và không gian nghiệm có số chiều 2
Trị riêng 1 có bội 1 và không gian nghiệm có số chiều 1

Bước 3: Xây dựng ma trận P và D
Ma trận P (các cột là vector riêng):
[-1.0, -1.0, 1.0]
[1.0, 0, -1.0]
[0, 1.0, 1.0]

Ma trận P^-1:
[1.0, 2.0, 1.0]
[-1.0, -1.0, 0.0]
[1.0, 1.0, 1.0]

Ma trận đường chéo D:
[-2, 0, 0]
[0, -2, 0]
[0, 0, 1]

Kiểm tra ma trận A = P*D*P^-1:
[1.0, 3.0, 3.0]
[-3.0, -5.0, -3.0]
[3.0, 3.0, 1.0]


Ví dụ 2:

In [75]:
A = [[1, -1], [2, 4]]

print("Ma trận A:")
print_matrix(A)
print()

P, P_inverse, D = is_diagonalizable(A)

if P is not None:
    print("Ma trận P (các cột là vector riêng):")
    print_matrix(P)
    
    print("\nMa trận P^-1:")
    print_matrix(P_inverse)
    
    print("\nMa trận đường chéo D:")
    print_matrix(D)
    
    print("\nKiểm tra ma trận A = P*D*P^-1:")
    result = matrix_multiply(P, matrix_multiply(D, P_inverse))
    print_matrix(result)
else:
    print("\nMa trận không thể chéo hóa được.")

Ma trận A:
[1, -1]
[2, 4]

Bước 1: Tính đa thức đặc trưng
Hệ số của đa thức đặc trưng (theo lũy thừa tăng dần): [1, -5, 6]

Bước 2: Tìm trị riêng và vector riêng
Trị riêng và bội số tương ứng: {3: 1, 2: 1}
Trị riêng 3 có bội 1 và không gian nghiệm có số chiều 1
Trị riêng 2 có bội 1 và không gian nghiệm có số chiều 1

Bước 3: Xây dựng ma trận P và D
Ma trận P (các cột là vector riêng):
[-0.5, -1.0]
[1.0, 1.0]

Ma trận P^-1:
[2.0, 2.0]
[-2.0, -1.0]

Ma trận đường chéo D:
[3, 0]
[0, 2]

Kiểm tra ma trận A = P*D*P^-1:
[1.0, -1.0]
[2.0, 4.0]


***
## **6. Mở rộng: Sử dụng thư viện và so sánh kết quả**

Ta có thể sử dụng các hàm trong thư viện NumPy:
-  Sử dụng np.linalg.eig() để tính trị riêng và vector riêng
- np.linalg.inv() để tính nghịch đảo ma trận

In [76]:
import numpy as np

A = np.array([[1, 3, 3], [-3, -5, -3], [3, 3, 1]])

print("Ma trận A:")
print(A)
print()

# Tính trị riêng và vector riêng
eigenvalues, eigenvectors = np.linalg.eig(A)

# Kiểm tra xem ma trận có chéo hóa được không
if len(eigenvectors) == A.shape[0] and np.linalg.matrix_rank(eigenvectors) == A.shape[0]:
    P = eigenvectors  # Ma trận P chứa các vector riêng theo cột
    P_inverse = np.linalg.inv(P)  # Tính ma trận nghịch đảo của P
    D = np.diag(eigenvalues)  # Ma trận đường chéo với các trị riêng
    
    print("Ma trận P (các cột là vector riêng):")
    print(P)
    
    print("\nMa trận P^-1:")
    print(P_inverse)
    
    print("\nMa trận đường chéo D:")
    print(D)
    
    print("\nKiểm tra ma trận A = P*D*P^-1:")
    result = P @ D @ P_inverse  # Nhân ma trận P, D và P^-1
    print(result)
    
    # Kiểm tra sai số giữa A và P*D*P^-1
    error = np.max(np.abs(A - result))
    print(f"\nSai số tối đa: {error}")
else:
    print("\nMa trận không thể chéo hóa được.")

Ma trận A:
[[ 1  3  3]
 [-3 -5 -3]
 [ 3  3  1]]

Ma trận P (các cột là vector riêng):
[[-0.57735027 -0.78762616  0.42064462]
 [ 0.57735027  0.20744308 -0.81636981]
 [-0.57735027  0.58018308  0.3957252 ]]

Ma trận P^-1:
[[-1.73205081 -1.73205081 -1.73205081]
 [-0.75691664 -0.04484052  0.71207612]
 [-1.41727082 -2.46126427 -1.04399344]]

Ma trận đường chéo D:
[[ 1.  0.  0.]
 [ 0. -2.  0.]
 [ 0.  0. -2.]]

Kiểm tra ma trận A = P*D*P^-1:
[[ 1.  3.  3.]
 [-3. -5. -3.]
 [ 3.  3.  1.]]

Sai số tối đa: 1.3322676295501878e-15


Ngoài ra, ta cũng có thể sử dụng thư viện `sympy` với hàm eigenvects() để tính trị riêng và vector riêng và hàm inv() để tìm nghịch đảo của ma trận

In [77]:
import sympy as sp

A = sp.Matrix([[1, 3, 3], [-3, -5, -3], [3, 3, 1]])

print("Ma trận A:")
print(A)
print()

# Tính trị riêng và vector riêng
eigensystem = A.eigenvects()

# eigensystem trả về danh sách các bộ 3: (trị riêng, bội số, danh sách vector riêng)
eigenvalues = []
eigenvectors = []

for eigenvalue, multiplicity, basis in eigensystem:
    for _ in range(multiplicity):
        eigenvalues.append(eigenvalue)
    for vector in basis:
        eigenvectors.append(vector)

# Tạo ma trận P từ các vector riêng
P = sp.Matrix.hstack(*eigenvectors)

# Kiểm tra xem ma trận có chéo hóa được không
if P.shape[1] == A.shape[0] and P.rank() == A.shape[0]:
    P_inverse = P.inv()
    
    # Tạo ma trận đường chéo D từ các trị riêng
    D = sp.diag(*eigenvalues)
    
    print("Ma trận P (các cột là vector riêng):")
    print(P)
    
    print("\nMa trận P^-1:")
    print(P_inverse)
    
    print("\nMa trận đường chéo D:")
    print(D)
    
    print("\nKiểm tra ma trận A = P*D*P^-1:")
    result = P * D * P_inverse
    print(result)
    
    # Kiểm tra sai số giữa A và P*D*P^-1
    is_equal = A == result
    print(f"\nA = P*D*P^-1: {is_equal}")
else:
    print("\nMa trận không thể chéo hóa được.")

Ma trận A:
Matrix([[1, 3, 3], [-3, -5, -3], [3, 3, 1]])

Ma trận P (các cột là vector riêng):
Matrix([[-1, -1, 1], [1, 0, -1], [0, 1, 1]])

Ma trận P^-1:
Matrix([[1, 2, 1], [-1, -1, 0], [1, 1, 1]])

Ma trận đường chéo D:
Matrix([[-2, 0, 0], [0, -2, 0], [0, 0, 1]])

Kiểm tra ma trận A = P*D*P^-1:
Matrix([[1, 3, 3], [-3, -5, -3], [3, 3, 1]])

A = P*D*P^-1: True


**Nhận xét**:
- Kết quả dùng thư viện `sympy` gần tương đồng với kết quả gốc do cài đặt thủ công hơn so với kết quả dùng thư viện `numpy`.
- `sympy` thực hiện các tính toán chính xác hơn nên không bị vấn đề về sai số số học như `numpy`.
- `sympy` có xu hướng chạy chậm hơn so với `numpy`, vì `sympy` tập trung vào tính toán biểu tượng (symbolic) chính xác thay vì tính toán số (numerical) hiệu quả.


***

## **7. Mở rộng: Ứng dụng của chéo hóa**


> Phép chéo hóa ma trận (diagonalization) là một quá trình quan trọng trong đại số tuyến tính với nhiều ứng dụng thực tiễn. Ứng dụng của ma trận chéo hóa rất đa dạng, từ việc giải quyết các bài toán trong cơ học lượng tử, xử lý tín hiệu, đến việc phân tích hệ thống động lực học. Các ma trận chéo hóa giúp tối ưu hóa các phép tính và cải thiện hiệu quả của các thuật toán. Dưới đây là một số ứng dụng tiêu biểu:

#### **a. Tính lũy thừa của ma trận**
Để tìm A^n, ta chéo hóa ma trận A trên, vì nếu A chéo hóa được thì:

> (P^−1)AP = D ⇔ A = PD(P^−1)  
⇔ A^k = (PD(P^−1))^k = PD^k(P^−1)

*D^k rất dễ tính vì chỉ cần lấy lũy thừa của các phần tử trên đường chéo.*

Tiếp cận này có thể được tổng quát hóa lên với hàm mũ ma trận và các hàm ma trận khác mà có thể được định nghĩa theo chuỗi lũy thừa.

#### **b. Tìm một dãy số thỏa công thức truy hồi**

Ví dụ như dãy Fibonacci: Mỗi số hạng trong dãy Fibonnacci, kể từ số hạng thứ ba, bằng tổng của hai số hạng đứng ngay trước nó. Ta có thể dùng chéo hóa để tìm công thức tổng quát để xác định số hạng F_n.

#### **c. Phân tích thành phần chính (PCA)**

Trong phân tích dữ liệu, PCA sử dụng chéo hóa ma trận hiệp phương sai để xác định các thành phần chính - những hướng có phương sai lớn nhất trong dữ liệu. Các vector riêng tương ứng với trị riêng lớn nhất chỉ ra các hướng quan trọng nhất trong không gian dữ liệu, từ đó giảm chiều dữ liệu và loại bỏ nhiễu. Điều này cải thiện hiệu quả và độ chính xác của các thuật toán học máy trong khoa học dữ liệu và học máy. Điều này còn được ứng dụng trong nhận diện khuôn mặt.

#### **d. Giải hệ phương trình vi phân**

Chéo hóa ma trận được sử dụng để giải các hệ phương trình vi phân, đặc biệt là trong các mô hình toán học phức tạp như động lực học chất lỏng, hệ thống cơ học, và mô phỏng khí hậu.

#### **e. Xử lý tín hiệu và hình ảnh**

Trong lĩnh vực xử lý tín hiệu và hình ảnh, việc chéo hóa ma trận giúp phân tích các tín hiệu và hình ảnh dễ dàng hơn. Ví dụ, trong nén ảnh, ma trận chéo hóa giúp giảm lượng thông tin cần lưu trữ mà vẫn giữ được chất lượng ảnh.

#### **f. Tối ưu hóa bộ nhớ và tính toán**

Ma trận chéo hóa giúp giảm số lượng phép tính cần thiết khi thực hiện các phép toán ma trận, từ đó tối ưu hóa bộ nhớ và tăng tốc độ tính toán. Điều này đặc biệt hữu ích trong các ứng dụng yêu cầu xử lý dữ liệu lớn.

#### **g. Định tuyến mạng**
Trong lĩnh vực mạng máy tính, ma trận chéo hóa được sử dụng để tối ưu hóa các thuật toán định tuyến, giúp cải thiện hiệu quả truyền tải dữ liệu qua mạng.

#### **h. Chéo hóa trong cơ học lượng tử**
Trong các tính toán của cơ học lượng tử và hóa lượng tử, chéo hóa ma trận là một trong những quy trình số thường được áp dụng nhất. Lý do cơ bản là do phương trình không phụ thuộc thời gian Schrödinger là một phương trình giá trị riêng, mặc dù nó là trên một không gian vô hạn chiều (một không gian Hilbert) trong hầu hết các tình huống vật lý.


***
## **8. Mô tả ý tưởng và các hàm**
### **Ý tưởng**

Cho A là ma trận (vuông) có thể chéo hóa được. Chương trình sẽ triển khai thuật toán chéo hóa ma trận để tìm ra ma trận làm chéo P, P^(-1) và ma trận đường chéo D:
- P là ma trận có các cột là các vector riêng của A
- P^(-1) là ma trận nghịch đảo của P (*sử dụng thuật toán Gauss - Jordan*)
- D là ma trận đường chéo chứa các trị riêng tương ứng

Thuật toán chéo hóa đã được trình bày ở Phần 2.

Để tìm trị riêng từ đa thức đặc trưng, tức là tìm nghiệm của phương trình đa thức đặc trưng bằng 0, ta kết hợp nhiều phương pháp để tìm ra nghiệm tổng quát với bậc n bất kỳ (n = 1, 2, 3, 4, ...):
  - Áp dụng các công thức trực tiếp cho đa thức bậc 1, 2 (delta)
  - Với các phương trình bậc cao hơn, ta áp dụng phương pháp chia đôi (bisection) để xác định vùng chứa nghiệm, áp dụng phương pháp Newton để tinh chỉnh nghiệm

### **Mô tả các hàm**

#### **a. Các hàm tiện ích cơ bản**
- `create_matrix(rows, cols, default_value=0)`: Tạo ma trận với kích thước `rows`, `cols`
- `create_identity_matrix(n)`: Tạo ma trận đơn vị kích thước n × n
- `matrix_copy(A)`: Tạo bản sao ma trận A
- `matrix_multiply(A, B)`: Nhân hai ma trận
- `matrix_subtract(A, B)`: Trừ hai ma trận
- `num_multiply(num, A)`: Nhân ma trận với một số
- `matrix_subtract_lambda_I(A, lambda_val)`: Tạo ma trận A - λI
- `polynomial_multiply(p1, p2)`: Nhân hai đa thức
- `determinant(A)`: Tính định thức ma trận bằng khai triển theo cột
- `round_matrix(A, decimals)`: Làm tròn các phần tử ma trận
- `print_matrix(A)`: In ra ma trận

#### **b. Tính đa thức đặc trưng**
- `get_minor_matrix(matrix, rows_to_remove, cols_to_remove)`: Tạo ma trận con
- `characteristic_polynomial(A)`: Tính đa thức đặc trưng P(λ) = det(A - λI)
  - Xử lý riêng cho ma trận kích thước 1×1, 2×2, 3×3
  - Tính tổng quát cho ma trận n × n bằng phương pháp khai triển
  - Trả về hệ số đa thức theo bậc giảm dần

#### **c. Tìm trị riêng**
- `find_eigenvalues(poly)`: Tìm trị riêng và bội số từ đa thức đặc trưng
- `find_polynomial_roots_with_multiplicity(poly)`: Tìm nghiệm và bội số của đa thức
- `polynomial_deflation(poly, root)`: Chia đa thức cho (x - root) bằng thuật toán Horner để tiếp tục tìm các nghiệm mới
- `evaluate_polynomial(poly, x)`: Tính giá trị đa thức tại x
- `find_real_roots_with_bisection(poly)`: Tìm nghiệm thực bằng phương pháp chia đôi
- `bisection_method(poly, a, b)`: Tìm nghiệm trong khoảng [a,b] bằng phương pháp chia đôi
- `newton_method(poly, x0)`: Tìm nghiệm bằng phương pháp Newton
- `derivative_polynomial(poly)`: Tính đạo hàm của đa thức

#### **d. Tìm vector riêng**
- `gauss_elimination(A)`: Đưa ma trận về dạng bậc thang rút gọn
- `find_eigenvectors(A, eigenval, multiplicity)`: Tìm vector riêng tương ứng với trị riêng
  - Giải hệ phương trình (A - λI)x = 0
  - Xác định biến tự do và pivot
  - Tính toán giá trị của các biến phụ thuộc

#### **e. Tính ma trận nghịch đảo**
- `compute_inverse(P)`: Tính ma trận nghịch đảo bằng phương pháp Gauss - Jordan với ma trận mở rộng [P|I].

#### **f. Chéo hóa ma trận**
- `is_diagonalizable(A)`: Kiểm tra và thực hiện chéo hóa ma trận A
  - Trả về ma trận P, P⁻¹ và D nếu ma trận chéo hóa được
  - Trả về cảnh báo, thông báo nếu không chéo hóa được

#### **g. So sánh với thư viện**
- Sử dụng `numpy.linalg.eig()` để tính trị riêng và vector riêng
- Sử dụng `sympy.Matrix.eigenvects()` để tính trị riêng và vector riêng với độ chính xác cao hơn, gần với đáp án do cài đặt thủ công hơn